In [ ]:
!pip install pyspark >> None

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, when, max, month, year
from pyspark.sql.functions import countDistinct
from pyspark.sql import functions as F
import pandas as pd

###Задание 1. Напишите программу, которая считывает файл с данными о продажах товаров и вычисляет общую сумму продаж, среднюю сумму продажи и максимальную сумму продажи за каждый месяц. Отсортировать по месяцам. Результаты вывести на экран.

In [ ]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

df_pyspark = spark.read.load('sales_data.csv', format='csv', sep=',', header='true', Infer_schema=True)

df_pyspark = df_pyspark.withColumn('order_id', df_pyspark['order_id'].cast('int'))
df_pyspark = df_pyspark.withColumn('product_id', df_pyspark['product_id'].cast('int'))
df_pyspark = df_pyspark.withColumn('customer_id', df_pyspark['customer_id'].cast('int'))
df_pyspark = df_pyspark.withColumn('order_date', df_pyspark['order_date'].cast('date'))
df_pyspark = df_pyspark.withColumn('quantity', df_pyspark['quantity'].cast('int'))
df_pyspark = df_pyspark.withColumn('price_per_unit', df_pyspark['price_per_unit'].cast('int'))
df_pyspark = df_pyspark.withColumn('total_price', df_pyspark['total_price'].cast('int'))

df_pyspark = df_pyspark.withColumn('order_month', month(df_pyspark['order_date']))

sales_analysis = df_pyspark.groupBy('order_month').agg(
    sum('total_price').alias('total_sales'),
    avg('total_price').alias('average_sales'),
    max('total_price').alias('max_sales'),
)
sales_analysis.orderBy('order_month').show()

+-----------+-----------+-----------------+---------+
|order_month|total_sales|    average_sales|max_sales|
+-----------+-----------+-----------------+---------+
|          1|       1300|            650.0|      700|
|          2|       2300|766.6666666666666|     1000|
|          3|       4500|            900.0|     1100|
|          4|       3350|            670.0|     1200|
|          5|       3800|            950.0|     1500|
|          6|       4650|            775.0|     1350|
|          7|       3700|            925.0|     1400|
|          8|       2350|783.3333333333334|      900|
|          9|       1450|            725.0|      900|
|         10|       1350|            675.0|     1200|
|         11|       2300|           1150.0|     1350|
|         12|       1400|            700.0|      800|
+-----------+-----------+-----------------+---------+



###Задание 2. Вычислите количество товаров, купленных различными методами оплаты.

In [ ]:
sales_analysis = df_pyspark.groupBy('payment_method').agg(sum('quantity').alias('num_of_sales'))
sales_analysis.show()

+--------------+------------+
|payment_method|num_of_sales|
+--------------+------------+
|      Наличные|          26|
|         Карта|          47|
+--------------+------------+



###Задание 3. Найдите регион с самой большой суммарной стоимостью продаж.

In [ ]:
sales_analysis = df_pyspark.groupBy('region').agg(sum('total_price').alias('total_price_per_region'))
total_price_per_region = sales_analysis.select('region').orderBy(sales_analysis.total_price_per_region.desc()).first()
print(*total_price_per_region)

Север


###Задание 4. Вычислите общую сумму продаж и среднюю сумму продаж для каждого региона.

In [ ]:
sales_analysis = df_pyspark.groupBy('region').agg(sum('total_price').alias('total_price_per_region'), avg('total_price').alias('avg_price_per_region'))
sales_analysis.show()

+------+----------------------+--------------------+
|region|total_price_per_region|avg_price_per_region|
+------+----------------------+--------------------+
|    Юг|                  8950|   813.6363636363636|
| Запад|                  4800|   533.3333333333334|
| Север|                 10200|   927.2727272727273|
|Восток|                  8500|   944.4444444444445|
+------+----------------------+--------------------+



###Задание 5. Вычислите общее количество и сумму товаров, проданных за наличные в 2022 году.

In [ ]:
df_pyspark = df_pyspark.withColumn('order_year', year(df_pyspark['order_date']))

sales_analysis = df_pyspark.filter((df_pyspark.payment_method == 'Наличные') & (df_pyspark.order_year == 2022)).agg(
    sum('quantity').alias('total_quantity_2022'), sum('total_price').alias('total_price_2022'))

sales_analysis.show()

+-------------------+----------------+
|total_quantity_2022|total_price_2022|
+-------------------+----------------+
|                 12|            4950|
+-------------------+----------------+



###Задание 6. Найдите уникальное количество покупателей за 2022 году.

In [ ]:
sales_analysis = df_pyspark.filter(df_pyspark.order_year == 2022)
sales_analysis = sales_analysis.select(countDistinct('customer_id').alias('unique_users'))

sales_analysis.show()

+------------+
|unique_users|
+------------+
|          16|
+------------+



###Задание 7. Вам даны данные с информацией о стоимости продуктов в различных валютах. Ваша задача состоит в том, чтобы перевести все цены в доллары, используя текущие курсы валют. Однако у вас есть ограничение: для некоторых продуктов курс валюты неизвестен и их стоимость должна остаться в исходной валюте. (Для конвертации из EUR в USD нужно умножить на 1.2)

In [ ]:
spark = SparkSession.builder.appName('currency_conversion').getOrCreate()

data = [(1, 100, 'USD'),
        (2, 200, 'EUR'),
        (3, 300, 'Unknown'),
        (4, 100, 'EUR'),
        (5, 100, 'USD'),
        (6, 300, 'Unknown'),
        (7, 100, 'Unknown'),
        (8, 200, 'USD'),
        (9, 300, 'USD'),]

df = spark.createDataFrame(data, ['product_id', 'price', 'currency'])

df = df.withColumn('price_usd', when(df.currency == 'USD', df.price).
                   when(df.currency == 'EUR', df.price * 1.2).
                   otherwise(df.price))

df.show()

+----------+-----+--------+---------+
|product_id|price|currency|price_usd|
+----------+-----+--------+---------+
|         1|  100|     USD|    100.0|
|         2|  200|     EUR|    240.0|
|         3|  300| Unknown|    300.0|
|         4|  100|     EUR|    120.0|
|         5|  100|     USD|    100.0|
|         6|  300| Unknown|    300.0|
|         7|  100| Unknown|    100.0|
|         8|  200|     USD|    200.0|
|         9|  300|     USD|    300.0|
+----------+-----+--------+---------+



###Задание 8. Допустим, есть два датасета: один содержит информацию о пользователях (user_id, name, age), а другой содержит информацию о покупках пользователей (user_id, product_id, date). Необходимо найти средний возраст пользователей, совершивших покупки.

In [ ]:
spark = SparkSession.builder.appName('user_purchase_join').getOrCreate()

users_df = spark.createDataFrame([
        (1, 'Alice', 25),
        (2, 'Bob', 30),
        (3, 'Charlie', 28),
        (4, 'John', 56),
        (5, 'Alex', 41),
        (6, 'Julia', 17)], ['user_id', 'name', 'age'])

purchases_df = spark.createDataFrame([
        (1, 101, '2022-01-01'),
        (2, 102, '2022-01-02'),
        (3, 103, '2022-01-03'),
        (3, 104, '2022-01-04'),
        (6, 105, '2022-01-05')], ['user_id', 'product_id', 'date'])

result_df = users_df.join(purchases_df, 'user_id').groupBy('user_id').agg(avg('age').alias('average_age'))

result_df.show()

+-------+-----------+
|user_id|average_age|
+-------+-----------+
|      1|       25.0|
|      2|       30.0|
|      3|       28.0|
|      6|       17.0|
+-------+-----------+



###Задание 9. У вас есть два набора данных. Первый набор содержит информацию о продуктах: id продукта, название, категория и цена. Второй набор содержит информацию о заказах: id заказа, id продукта, количество. Ваша задача - использовать PySpark для выполнения следующих шагов:

1. Присоединить набор данных о продуктах к набору данных о заказах с помощью id продукта.
2. Рассчитать общую стоимость заказа, учитывая количество продуктов и их цену.
3. Отфильтровать заказы, у которых общая стоимость больше 1000.

In [ ]:
spark = SparkSession.builder.appName('aggregate-join-filter').getOrCreate()

data_products = [(1, 'product1', 'category1', 10.0),
                 (2, 'product2', 'category2', 15.0),
                 (3, 'product3', 'category1', 12.5),
                 (4, 'product4', 'category3', 20.0),
                 (5, 'product5', 'category2', 18.0),
                 (6, 'product6', 'category3', 25.0),
                 (7, 'product7', 'category1', 9.0),
                 (8, 'product8', 'category2', 16.0),
                 (9, 'product9', 'category3', 22.0),
                 (10, 'product10', 'category1', 11.5)]

products_df = spark.createDataFrame(data_products, ['product_id', 'title', 'category', 'price'])

data_orders = [(1, 1, 55),
               (2, 3, 206),
               (3, 2, 321),
               (4, 5, 123),
               (5, 4, 47),
               (6, 7, 27),
               (7, 6, 38),
               (8, 8, 21),
               (9, 10, 111),
               (10, 9, 48)]

orders_df = spark.createDataFrame(data_orders, ['order_id', 'product_id', 'quantity'])

joined_df = orders_df.join(products_df, 'product_id')

total_cost_df = joined_df.withColumn('total_cost', F.col('quantity') * F.col('price'))

filtered_orders_df = total_cost_df.filter(total_cost_df.total_cost > 1000)

filtered_orders_df.show()

+----------+--------+--------+---------+---------+-----+----------+
|product_id|order_id|quantity|    title| category|price|total_cost|
+----------+--------+--------+---------+---------+-----+----------+
|         2|       3|     321| product2|category2| 15.0|    4815.0|
|         3|       2|     206| product3|category1| 12.5|    2575.0|
|         5|       4|     123| product5|category2| 18.0|    2214.0|
|         9|      10|      48| product9|category3| 22.0|    1056.0|
|        10|       9|     111|product10|category1| 11.5|    1276.5|
+----------+--------+--------+---------+---------+-----+----------+



###Задание 10. Найти сумму чисел в колонке.

In [ ]:
spark = SparkSession.builder.appName('sum_example').getOrCreate()

data = [(1,), (2,), (3,), (4,)]
df = spark.createDataFrame(data, ['number'])

sum_result = df.select(sum('number')).show()

+-----------+
|sum(number)|
+-----------+
|         10|
+-----------+



###Задание 11. Посчитать количество уникальных значений в колонке.

In [ ]:
spark = SparkSession.builder.appName('count_distinct_example').getOrCreate()

data = [('Alice',), ('Bob',), ('Alice',), ('Eve',)]
df = spark.createDataFrame(data, ['name'])

count_result = df.select(countDistinct('name')).show()

+--------------------+
|count(DISTINCT name)|
+--------------------+
|                   3|
+--------------------+



###Задание 12. Выполнить фильтрацию данных по определённому условию.

In [ ]:
spark = SparkSession.builder.appName('filter_example').getOrCreate()

data = [('Alice', 25), ('Bob', 30), ('Eve', 20), ('Charlie', 35)]
df = spark.createDataFrame(data, ['name', 'age'])

filtered_data = df.filter(countDistinct('name')).show()

+-----+---+
| name|age|
+-----+---+
|Alice| 25|
|  Eve| 20|
+-----+---+

